In [ ]:
from ipycanvas import Canvas, hold_canvas
from Klasser.celle import *
from Klasser.busk import *
import random
from time import sleep


# visual size of tiles
tile_size = 15
visual_space = 2
rect_size = tile_size - visual_space

# amount of tiles in each direction
tiles_hori = 10
tiles_vert = 7

antal_celler = 1
start_energi = 10

antal_busker = 2

# Contains objects inhabiting the world
entities = []

# Create empty world, to contain references to cells
verd = []
for x in range(tiles_hori):
    row = []
    for y in range(tiles_vert):
        row.append(None)
    verd.append(row)

# Populate world
celler_igjen = antal_celler
busker_igjen = antal_busker
while celler_igjen > 0 or busker_igjen > 0:
    x = random.randint(0, tiles_hori - 1)
    y = random.randint(0, tiles_vert - 1)
    while verd[x][y] != None:
        x = random.randint(0, tiles_hori - 1)
        y = random.randint(0, tiles_vert - 1)
    if celler_igjen > 0:
        new_cell = Celle(x, y, start_energi, [])
        entities.append(new_cell)
        verd[x][y] = len(entities) - 1
        celler_igjen -= 1
    else:
        new_busk = Busk(x, y, 5, 5)
        entities.append(new_busk)
        verd[x][y] = len(entities) - 1
        busker_igjen -= 1


# Canvas size
width = tile_size * tiles_hori
height = tile_size * tiles_vert

canvas = Canvas(height = height, width = width)
display(canvas)

def draw_world():
    # Draw grid
    canvas.fill_style = "gray"
    for x in range(tiles_hori):
        for y in range(tiles_vert):
            canvas.fill_rect(x * tile_size, y * tile_size, rect_size, rect_size)

def rollover(list, index, radius):
    if index - radius < 0 or index + radius >= len(list) - 1:
        return list[index - radius:] + list[:(index + radius + 1) % len(list)]
    else:
        return list[index - radius : index + radius + 1]

def get_nabolag(pos_x, pos_y, radius):
    nabolag = []
    nabolag_rad = []
    for y in range(pos_y - radius, pos_y + radius + 1):
        for x in range(pos_x - radius, pos_x + radius + 1):
            nabolag_rad.append(verd[x % tiles_hori][y % tiles_vert])
        nabolag.append(nabolag_rad)
        nabolag_rad = []
    return nabolag

#Løkke som repeterast til programmet er ferdig
for i in range(100):
    for j in range(len(entities)):
        entity = entities[j]
        if type(entity) == Busk:
            entity.veks()
        elif type(entity) == Celle:
            nabolag = get_nabolag(entity.pos_x, entity.pos_y, 2)
            entity.finn_mål(nabolag, entities)
            beveg_x, beveg_y = entity.beveg()
            # Move the cell towards its target if there isnt anything in the way
            if verd[beveg_x % tiles_hori][beveg_y % tiles_vert] == None:
                verd[entity.pos_x][entity.pos_y] = None
                entity.pos_x = beveg_x
                entity.pos_y = beveg_y
                verd[x][y] = j

    # Draw everything
    with hold_canvas():
        draw_world()
        for entity in entities:
            entity.tegn(canvas, tile_size, rect_size)
    
    sleep(0.5)